In [ ]:
# Check to see if we're using a GPU
!nvidia-smi

##101 food classes: working with less data
Our goal is to beath the original Food101 paper with 10% of the training data.

In [38]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-10-14 00:18:14--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 64.233.170.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip.1’

101_food_classes_10   1%[                    ]  24.72M  11.3MB/s               ^C


In [2]:
import zipfile
zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [3]:
# How many images in each folder?
import os

# Walk through -1 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("101_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in `{dirpath}`")

There are 2 directories and 0 images in `101_food_classes_10_percent`
There are 101 directories and 0 images in `101_food_classes_10_percent/train`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_toast`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/frozen_yogurt`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/baklava`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/donuts`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_onion_soup`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/bruschetta`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/ramen`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/cheesecake`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/hot_dog`
There are 0 directories and 75 images in `101_food_classes_10_p

In [39]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [62]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)  # don't shuffle test data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


In [23]:
class_names = train_data_all_10_percent.class_names

In [53]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Coverts image datatype from 'unit8' -> 'float32' and reshapes image to [img_shape, img_shape, colour_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape target image
  # image = image/255.  # scale image values (not required with EfficientNetBX models from tf.keras.applications)
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

In [55]:
# Map preprocessing function to training (and parallelize)
train_data_all_10_percent = train_data_all_10_percent.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data_all_10_percent.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [57]:
train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 101), dtype=tf.float32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 101), dtype=tf.float32, name=None))>)

In [15]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-10-13 23:37:44--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-10-13 23:37:44 (120 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [16]:
# Import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

In [17]:
# Create tensorboard callback (import from helper_functions.py)
from helper_functions import create_tensorboard_callback

# Create ModelCheckpoint callback to save a model's progress during training
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_acc",
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0) # don't print whether or not model is being saved

In [18]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [19]:
!nvidia-smi

Fri Oct 13 23:39:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    28W /  70W |    361MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
mixed_precision.global_policy()

<Policy "mixed_float16">

In [66]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if your model doesn't you can have a layer like below
# X = preprocessing.Rescaling(1./255)(x)
x = base_model(inputs, training=False)    # make sure layers which should be inference mode only stay like that
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype="float32", name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [26]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d_1  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense (Dense)               (None, 101)               129381    
                                                                 
 softmax_float32 (Activatio  (None, 101)               0         
 n)                                                              
                                                             

In [27]:
# Check the dtype_policy attributes of layers in our model
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_layer True float32 <Policy "float32">
efficientnetb0 False float32 <Policy "mixed_float16">
global_average_pooling2d_1 True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


In [28]:
# Check the dtype policy attributes of layers attributes of layer in the base model
for layer in model.layers[1].layers: # check the layers of the base model (layer at index 1 of `model`)
  print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_2 False float32 <Policy "float32">
rescaling_2 False float32 <Policy "mixed_float16">
normalization_1 False float32 <Policy "mixed_float16">
rescaling_3 False float32 <Policy "mixed_float16">
stem_conv_pad False float32 <Policy "mixed_float16">
stem_conv False float32 <Policy "mixed_float16">
stem_bn False float32 <Policy "mixed_float16">
stem_activation False float32 <Policy "mixed_float16">
block1a_dwconv False float32 <Policy "mixed_float16">
block1a_bn False float32 <Policy "mixed_float16">
block1a_activation False float32 <Policy "mixed_float16">
block1a_se_squeeze False float32 <Policy "mixed_float16">
block1a_se_reshape False float32 <Policy "mixed_float16">
block1a_se_reduce False float32 <Policy "mixed_float16">
block1a_se_expand False float32 <Policy "mixed_float16">
block1a_se_excite False float32 <Policy "mixed_float16">
block1a_project_conv False float32 <Policy "mixed_float16">
block1a_project_bn False float32 <Policy "mixed_float16">
block2a_expand_conv False float

In [68]:
## Fit the feature extraction model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data_all_10_percent,
                                                     epochs=10,
                                                     steps_per_epoch=(len(train_data_all_10_percent)),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                                                            experiment_name="efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

Saving TensorBoard log files to: training_logs/efficientnetb0_101_classes_all_data_feature_extract/20231014-004408
Epoch 1/10
236/237 [============================>.] - ETA: 0s - loss: 1.1059 - accuracy: 0.7554

237/237 [==============================] - 27s 110ms/step - loss: 1.1050 - accuracy: 0.7556 - val_loss: 1.6358 - val_accuracy: 0.5702
Epoch 2/10
237/237 [==============================] - ETA: 0s - loss: 0.9350 - accuracy: 0.7999

237/237 [==============================] - 28s 115ms/step - loss: 0.9350 - accuracy: 0.7999 - val_loss: 1.6076 - val_accuracy: 0.5760
Epoch 3/10
237/237 [==============================] - ETA: 0s - loss: 0.8008 - accuracy: 0.8367

237/237 [==============================] - 25s 102ms/step - loss: 0.8008 - accuracy: 0.8367 - val_loss: 1.5613 - val_accuracy: 0.5810
Epoch 4/10
237/237 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.8653

237/237 [==============================] - 27s 111ms/step - loss: 0.6930 - accuracy: 0.8653 - val_loss: 1.5758 - val_accuracy: 0.5776
Epoch 5/10
237/237 [==============================] - ETA: 0s - loss: 0.6031 - accuracy: 0.8923

...

237/237 [==============================] - 26s 109ms/step - loss: 0.5260 - accuracy: 0.9121 - val_loss: 1.5646 - val_accuracy: 0.5834
Epoch 7/10
236/237 [============================>.] - ETA: 0s - loss: 0.4614 - accuracy: 0.9306

237/237 [==============================] - 27s 113ms/step - loss: 0.4611 - accuracy: 0.9307 - val_loss: 1.5532 - val_accuracy: 0.5866
Epoch 8/10
237/237 [==============================] - ETA: 0s - loss: 0.4056 - accuracy: 0.9453

237/237 [==============================] - 28s 115ms/step - loss: 0.4056 - accuracy: 0.9453 - val_loss: 1.5487 - val_accuracy: 0.5879
Epoch 9/10
236/237 [============================>.] - ETA: 0s - loss: 0.3570 - accuracy: 0.9560

237/237 [==============================] - 26s 108ms/step - loss: 0.3570 - accuracy: 0.9559 - val_loss: 1.5550 - val_accuracy: 0.5877
Epoch 10/10
237/237 [==============================] - ETA: 0s - loss: 0.3140 - accuracy: 0.9671

237/237 [==============================] - 26s 108ms/step - loss: 0.3140 - accuracy: 0.9671 - val_loss: 1.5618 - val_accuracy: 0.5916
